In [ ]:
import tifffile as tiff
import cv2
import torch
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import torch.nn as nn

In [ ]:
# загрузка 5 каналов изображения, RGB и маски
img_B2 = tiff.imread('PATH_to_blue_channel_image')
img_B3 = tiff.imread('PATH_to_green_channel_image')
img_B4 = tiff.imread('PATH_to_red_channel_image')
img_B8 = tiff.imread('PATH_to_NIR_channel_image')
img_B12 = tiff.imread('PATH_to_SWIR_channel_image')
img_real = tiff.imread('PATH_to_true_color_image')
mask = cv2.imread('PATH_to_MASK')

In [ ]:
# функция перевода маски в категориальный вид
def image_cat(image, class_num, black_color = 128):
  pic = np.array(image)
  img = np.zeros((pic.shape[0], pic.shape[1], class_num))
  np.place(img[ :, :, 0], pic[ :, :, 0] >= black_color, 1)
  np.place(img[ :, :, 0], pic[ :, :, 2] >= black_color, 2)
  return img

In [ ]:
# создание 5-канального изображения, предобработка и загрузка данных
t = np.concatenate((img_B2[:, :, np.newaxis],
                    img_B3[:, :, np.newaxis],
                    img_B4[:, :, np.newaxis],
                    img_B8[:, :, np.newaxis],
                    img_B12[:, :, np.newaxis]), axis = 2)

t1 = A.Compose([
    A.Resize(256,256),
    ToTensorV2()
])

aug = t1(image=t)
t = np.array(aug['image'])
t = np.rollaxis(t, 0, 3)[np.newaxis,:]
mask = image_cat(mask, 1, black_color = 128)
aug = t1(image=mask)
mask = np.array(aug['image'])
mask = np.rollaxis(mask, 0, 3)[np.newaxis,:]

test_b = torch.utils.data.DataLoader(list(zip(np.rollaxis(t, 3, 1), mask)),
                                         batch_size=1, shuffle=False, pin_memory=True)

In [ ]:
# загрузка модели
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.jit.load('PATH_to_MODEL')
model.eval()

In [ ]:
# визуализация результатов
for x, y in test_b:
  x = x.to(DEVICE)
  fig , ax =  plt.subplots(1, 3, figsize=(12, 12))
  softmax = nn.Softmax(dim=1)
  preds = torch.argmax(softmax(model(x)),axis=1).to('cpu')
  img1 = np.transpose(np.array(x[0,:,:,:].to('cpu')),(1,2,0))
  [A1,A2,A3,A4,A5] = np.split(img1,[1,2,3,4], axis=2)
  img1 = A1.squeeze(2) + A2.squeeze(2) + A3.squeeze(2) + A4.squeeze(2) + A5.squeeze(2)
  preds1 = np.array(preds[0,:,:])
  mask1 = np.array(y[0,:,:])
  ax[0].set_title('Real image')
  ax[1].set_title('5-channel visualization')
  ax[2].set_title('Prediction')
  ax[0].axis("off")
  ax[1].axis("off")
  ax[2].axis("off")
  ax[0].imshow(img_real)
  ax[1].imshow(img1)
  ax[2].imshow(preds1)
  break